# Process SASSIE ocean model granules

In [374]:
import tarfile
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc4
from datetime import datetime, timedelta

## import ECCO utils
import sys
sys.path.append('/Users/mzahn/github_others/ECCOv4-py')
import ecco_v4_py as ecco

ECCO github docs: https://github.com/ECCO-GROUP/ECCOv4-py/blob/master/ecco_v4_py/netcdf_product_generation.py

*Info from Ian/Mike:*

TIME<br>
data.cal start time is 1992-01-01<br>
Start time of the model is 5790000 (22.0319 years after 1992-01-01)<br>
Which is 2014-01-06T16:00:00<br>
Model simulation goes to around end of 2021<br>

FILES<br>
Each gz file has 14 files, each for one day<br>
199 (files)*14 (days per gz file) = 2,786 (days)/365 (days per year)= 7.6329 years

From *.meta file:
 nDims = [   2 ];
 dimList = [
         40,         1,        40,
     102600,         1,    102600
 ];
 dataprec = [ 'float32' ];
 nrecords = [          3 ];
 timeStepNumber = [    5810400 ];
 timeInterval = [  6.971616000000E+08  6.972480000000E+08 ];
 missingValue = [ -9.99000000000000E+02 ];
 nFlds = [    3 ];
 fldList = {
 'SIarea  ' 'SIheff  ' 'SIhsnow '
 };


## ECCO routines 

### routines to convert between SASSIE N1 faces and compact

In [131]:
def load_sassie_N1_field(file_dir, fname, nk=1, skip=0):
    num_cols = 680*4 + 1080
    num_rows = 1080
    
    time_level = int(fname.split('.data')[0].split('.')[-1])
    
    tmp_compact = ecco.load_binary_array(file_dir, fname, \
                                    num_rows, num_cols, nk=nk, skip=skip, filetype='>f4')

    return tmp_compact, time_level

In [9]:
def sassie_n1_compact_to_faces_2D(sassie_n1_compact):
    sassie_faces = dict()
    n = 680
    
    # Face 1 
    start_row = 0
    end_row = n
    sassie_faces[1] = sassie_n1_compact[start_row:end_row,:]

    # Face 2
    start_row = end_row
    end_row = start_row + n

    sassie_faces[2] = sassie_n1_compact[start_row:end_row,:]
    
    # Face 3
    start_row = end_row
    end_row = start_row + 1080
    sassie_faces[3] = sassie_n1_compact[start_row:end_row:,:]
    
    #Face 4
    start_row = end_row
    end_row = end_row + 680
    sassie_faces[4] = sassie_n1_compact[start_row:end_row].reshape(1080, n)

    #Face 5
    start_row = end_row
    end_row = end_row + 680
    sassie_faces[5] = sassie_n1_compact[start_row:end_row].reshape(1080, n)

    return sassie_faces

In [11]:
def sassie_n1_compact_to_faces_3D(sassie_n1_compact):
    sassie_faces = dict()
    n = 680
    
    # Face 1 
    start_row = 0
    end_row = n
    sassie_faces[1] = sassie_n1_compact[:,start_row:end_row,:]

    # Face 2
    start_row = end_row
    end_row = start_row + n
    sassie_faces[2] = sassie_n1_compact[:,start_row:end_row,:]
    
    # Face 3
    start_row = end_row
    end_row = start_row + 1080
    sassie_faces[3] = sassie_n1_compact[:,start_row:end_row:,:]
    
    #Face 4
    start_row = end_row
    end_row = end_row + 680
    sassie_faces[4] = sassie_n1_compact[:,start_row:end_row].reshape(90, 1080, n)

    #Face 5
    start_row = end_row
    end_row = end_row + 680
    sassie_faces[5] = sassie_n1_compact[:,start_row:end_row].reshape(90, 1080, n)

    return sassie_faces

In [13]:
## function to slice small pieces from Faces 1 and 4 and combine them to the Arctic Face 3
def combine_sassie_N1_faces_to_HHv2_2D(face_arr):
    # dimensions of the final Arctic HH field. 535+185+1080=1800
    new_arr = np.zeros((1080, 1800)) 
    # cut out sections we want and assign them to location on HH
    new_arr[:, 185:185 + 1080] = face_arr[3]
    # rotate Face 1 to line up with orientation of Face 3
    new_arr[:, 0:185] = np.flipud(face_arr[1][-185:,:].T) # flip and transpose
    new_arr[:, 185 + 1080:] = face_arr[4][:,:535]

    new_arr = np.rot90(new_arr,2) # rotate it 180 so Greenland/AK are on bottom
    return new_arr

In [15]:
## function to slice small pieces from Faces 1 and 4 and combine them to the Arctic Face 3
def combine_sassie_N1_faces_to_HHv2_3D(face_arr):
    # dimensions of the final Arctic HH field. 535+185+1080=1800 ; 90 vertical levels
    new_arr = np.zeros((90, 1080, 1800)) 
    # cut out sections we want and assign them to location on HH
    new_arr[:, :, 185:185 + 1080] = face_arr[3]
    # rotate Face 1 to line up with orientation of Face 3
    new_arr[:, :, 0:185] = np.transpose(face_arr[1][:,-185:,::-1],axes=(0,2,1)) # flip and transpose
    new_arr[:, :, 185 + 1080:] = face_arr[4][:,:,:535]

    new_arr = np.rot90(new_arr,2,axes=(1,2)) # rotate it 180 so Greenland/AK are on bottom
    return new_arr

In [17]:
def plot_sassie_HHv2_3D(face_arr, depth_level=0, vmin=None, vmax=None,\
    cmap='jet', axs = None, \
    show_colorbar=True):

    tmp = combine_sassie_N1_faces_to_HHv2_3D(face_arr)

    if vmin == None:
        vmin = np.min(tmp)
    if vmax == None:
        vmax = np.max(tmp)

    if axs == None:
        plt.imshow(tmp[depth_level,:,:], origin='lower', interpolation='none',vmin=vmin,vmax=vmax, cmap=cmap)
        if show_colorbar:
            plt.colorbar()

    else:
        im1 = axs.imshow(tmp[depth_level,:,:], origin='lower', interpolation='none',vmin=vmin,vmax=vmax, cmap=cmap)
        fig = plt.gcf()
        if show_colorbar:
            fig.colorbar(im1, ax=axs)

In [29]:
def make_2D_HHv2_da(field_HH, model_grid_ds, da_name):
    tmp_da = xr.DataArray(field_HH, dims=['j','i'],\
                            coords={'XC': (('j','i'), model_grid_ds.XC.values),\
                                    'YC': (('j','i'), model_grid_ds.YC.values)})
    
    tmp_da.name = da_name
    # tmp_da = add_geo_metadata(tmp_da)
    
    return tmp_da

In [199]:
geometry_HHv2_ds = xr.Dataset(
    data_vars=dict(
        CS=(["j", "i"], angleCS_faces_HHv2),
        SN=(["j", "i"], angleSN_faces_HHv2),
        rAc=(["j","i"], rAc_HHv2),
        dxG=(["j_g","i"], dxG_HHv2),
        dyG=(["j","i_g"], dyG_HHv2),
    ),
    coords=dict(
        i   =(["i"], i_array),
        i_g =(["i_g"], i_g_array),
        j   =(["j"], j_array),
        j_g =(["j_g"], j_g_array),
        k   =(["k"], k_array),
        k_u =(["k_u"], k_u_array),
    ),
    attrs=dict(description="HHv2 model geometry data."),
)

In [561]:
def make_3D_HHv2_da(field_HH, model_grid_ds, timestamp, da_name='NAME', k_face='center'):
    if k_face == 'center':
        tmp_da = xr.DataArray([field_HH], dims=['time', 'k','j','i'],\
                                coords={'XC': (('j','i'), model_grid_ds.XC.values),\
                                        'YC': (('j','i'), model_grid_ds.YC.values),\
                                        'Z': (('k'), model_grid_ds.Z.values),\
                                        'Zu':(('k'),model_grid_ds.Zu.values),\
                                        'Zl':(('k'),model_grid_ds.Zl.values),\
                                        'time':(('time'),timestamp)})
        tmp_da['k'].attrs['axis']  = 'Z'

    elif k_face == 'top':
        tmp_da = xr.DataArray([field_HH], dims=['time','k_l','j','i'],\
                                coords={'XC': (('j','i'), model_grid_ds.XC.values),\
                                        'YC': (('j','i'), model_grid_ds.YC.values),\
                                        'Z': (('k'), model_grid_ds.Z.values),\
                                        'Zu':(('k'),model_grid_ds.Zu.values),\
                                        'Zl':(('k'),model_grid_ds.Zl.values),\
                                        'time':(('time'),timestamp)})
        tmp_da['k_l'].attrs['axis']  = 'Z'

    elif k_face == 'bottom':
        tmp_da = xr.DataArray([field_HH], dims=['time','k_u','j','i','time'],\
                                coords={'XC': (('j','i'), model_grid_ds.XC.values),\
                                        'YC': (('j','i'), model_grid_ds.YC.values),\
                                        'Z': (('k'), model_grid_ds.Z.values),\
                                        'Zu':(('k'),model_grid_ds.Zu.values),\
                                        'Zl':(('k'),model_grid_ds.Zl.values),\
                                        'time':(('time'),timestamp)})
        tmp_da['k_u'].attrs['axis']  = 'Z'


    tmp_da['Z'].attrs['long_name'] = 'grid cell depth at center'
    tmp_da['Zu'].attrs['long_name'] = 'grid cell depth at bottom'
    tmp_da['Zl'].attrs['long_name'] = 'grid cell depth at top'

    tmp_da['Z'].attrs['units'] = 'm'
    tmp_da['Zl'].attrs['units'] = 'm'
    tmp_da['Zu'].attrs['units'] = 'm'

    tmp_da.name = da_name
 #   tmp_da = add_geo_metadata(tmp_da)

    return tmp_da

In [534]:
def timestamp_from_iter_num(iter_num):
    """
    takes the model iteration that was pulled from the data's filename and converts it to its equivalent datetime
    """
    ## Start time of the model is 5790000 (22.0319 years after 1992-01-01)
    ## there are 120 seconds for each iteration and 86400 seconds per day
    ## take the iteration number, convert to seconds, and calculate number of days since start of model
    
    num_days_since_start = iter_num*120 / 86400 ## divide iter_number by 86400 which is equal to the number of seconds in a day
    model_start_time = datetime(1992,1,1) # data.cal start time is 1992-01-01
    
    timestamp = np.array([model_start_time + timedelta(days=num_days_since_start)], dtype='datetime64[ns]')
    
    return timestamp

In [591]:
def create_encoding(ecco_ds, output_array_precision = np.float32):
    
    # Create NetCDF encoding directives
    # ---------------------------------------------
    print('\n... creating variable encodings')
    # ... data variable encoding directives
    
    # Define fill values for NaN
    if output_array_precision == np.float32:
        netcdf_fill_value = nc4.default_fillvals['f4']

    elif output_array_precision == np.float64:
        netcdf_fill_value = nc4.default_fillvals['f8']
    
    dv_encoding = dict()
    for dv in ecco_ds.data_vars:
        dv_encoding[dv] =  {'zlib':True, \
                            'complevel':5,\
                            'shuffle':True,\
                            '_FillValue':netcdf_fill_value}

    # ... coordinate encoding directives
    print('\n... creating coordinate encodings')
    coord_encoding = dict()
    for coord in ecco_ds.coords:
        # set default no fill value for coordinate
        if output_array_precision == np.float32:
            coord_encoding[coord] = {'_FillValue':None, 'dtype':'float32'}
        elif output_array_precision == np.float64:
            coord_encoding[coord] = {'_FillValue':None, 'dtype':'float64'}

        # force 64 bit ints to be 32 bit ints
        if (ecco_ds[coord].values.dtype == np.int32) or \
           (ecco_ds[coord].values.dtype == np.int64) :
            coord_encoding[coord]['dtype'] ='int32'

        # fix encoding of time
        if coord == 'time' or coord == 'time_bnds':
            coord_encoding[coord]['dtype'] ='int32'

            if 'units' in ecco_ds[coord].attrs:
                # apply units as encoding for time
                coord_encoding[coord]['units'] = ecco_ds[coord].attrs['units']
                # delete from the attributes list
                del ecco_ds[coord].attrs['units']

        elif coord == 'time_step':
            coord_encoding[coord]['dtype'] ='int32'

    # ... combined data variable and coordinate encoding directives
    encoding = {**dv_encoding, **coord_encoding}

    return encoding

# Create routine to process files

In [27]:
sassie_n1_geometry_ds = xr.open_dataset('/Users/mzahn/data/SASSIE/GRID_GEOMETRY_SASSIE_HH_V1r1_native_llc1080.nc')

In [4]:
## open example file 
# targz_file = tarfile.open('./data/tar_gz_files/seaice_state_day_mean.0005810000.tar.gz') 
targz_file = tarfile.open('/Users/mzahn/data/SASSIE/SASSIE_examples/ocean_state_2D_day_mean/ocean_state_2D_day_mean.0005800000.tar.gz')

## extracting file to produce *.data and *.meta files
targz_file.extractall('/Users/mzahn/data/SASSIE/SASSIE_examples/ocean_state_2D_day_mean/data') 
targz_file.close() 

In [308]:
## model output directories
sassie_llc1080_data_dir = '/Users/mzahn/data/SASSIE/SASSIE_examples/'

## ocean 3D data directory
ocean_3d_file_dir = 'ocean_state_3D_day_mean/'
ocean_3d_file_subdir = 'ocean_state_3D_day_mean.0005790000/' # includes 14 data files from compressed file

## where to save netCDFs with directories for each variable
mds_output_dir = '/Users/mzahn/data/SASSIE/SASSIE_netcdfs/'

## identify data directories
data_dir = Path(sassie_llc1080_data_dir + ocean_3d_file_dir + ocean_3d_file_subdir) # using ocean 3d as first example
data_files = np.sort(list(data_dir.glob(mds_file + '*data')))

## identify number of fields in dataset
meta_files = np.sort(list(data_dir.glob(mds_file + '*meta')))
nFlds = [int(i) for i in meta_file[11].split('=')[-1].split() if i.isdigit()][0] # extract number from string
fldList = meta_file[13]

In [593]:
## loop through each time step ------

## loop through files
for file in data_files[0:2]:
    print('loading file: ', file)
    
    filename = str(file).split('/')[-1]
    
    if nFlds == 2:
        ## process binary data to compact format
        # nk=180 because there are two variables each with 90 vertical levels
        var1_compact, iter_num = load_sassie_N1_field(str(data_dir), filename, nk=90, skip=0)
        var2_compact, iter_num = load_sassie_N1_field(str(data_dir), filename, nk=90, skip=90)
        
        ## convert compact format to 5 faces
        var1_faces = sassie_n1_compact_to_faces_3D(data_compact[0:90,:,:]) # process first variable
        var2_faces = sassie_n1_compact_to_faces_3D(data_compact[90:180,:,:]) # process first variable
        
        ## convert faces to HHv2 Arctic rectangle
        var1_HHv2 = combine_sassie_N1_faces_to_HHv2_3D(var1_faces)
        var2_HHv2 = combine_sassie_N1_faces_to_HHv2_3D(var2_faces)
        
        ## Create DataArrays from HHv2
        var1_name = meta_file[13].split()[0].strip("' \t")
        var2_name = meta_file[13].split()[2].strip("' \t")
        
        ## add timestamp to dataset
        timestamp = timestamp_from_iter_num(iter_num)
        
        var1_HHv2_da = make_3D_HHv2_da(var1_HHv2, sassie_n1_geometry_ds, timestamp, da_name=var1_name, k_face='center')
        var2_HHv2_da = make_3D_HHv2_da(var2_HHv2, sassie_n1_geometry_ds, timestamp, da_name=var2_name, k_face='center')
        
        ## convert DataArrays to datasets
        var1_HHv2_ds = var1_HHv2_da.to_dataset()
        var2_HHv2_ds = var2_HHv2_da.to_dataset()

        # save netCDF files
        mds_output_dir_var1 = Path(mds_output_dir + ocean_3d_file_dir + var1_name)
        mds_output_dir_var2 = Path(mds_output_dir + ocean_3d_file_dir + var2_name)
        
        mds_output_dir_var1.mkdir(parents=True, exist_ok=True) # create output directory if it doesn't already exist
        mds_output_dir_var2.mkdir(parents=True, exist_ok=True)
        
        encoding_var1 = create_encoding(var1_HHv2_ds, output_array_precision = np.float32)
        encoding_var2 = create_encoding(var2_HHv2_ds, output_array_precision = np.float32)
        
        var1_filename_netcdf = "SASSIE_HH_" + var1_name + "_" + str(time_level) + ".nc"
        var2_filename_netcdf = "SASSIE_HH_" + var2_name + "_" + str(time_level) + ".nc"
        
        var1_HHv2_ds.to_netcdf(mds_output_dir_var1 / var1_filename_netcdf, encoding = encoding_var1)
        var2_HHv2_ds.to_netcdf(mds_output_dir_var2 / var2_filename_netcdf, encoding = encoding_var2)
        
        var1_HHv2_ds.close()
        var2_HHv2_ds.close()

loading file:  /Users/mzahn/data/SASSIE/SASSIE_examples/ocean_state_3D_day_mean/ocean_state_3D_day_mean.0005790000/ocean_state_3D_day_mean.0005796720.data
load_binary_array: loading file /Users/mzahn/data/SASSIE/SASSIE_examples/ocean_state_3D_day_mean/ocean_state_3D_day_mean.0005790000/ocean_state_3D_day_mean.0005796720.data
load_binary_array: data array shape  (180, 3800, 1080)
load_binary_array: data array type  >f4

... creating variable encodings

... creating coordinate encodings

... creating variable encodings

... creating coordinate encodings
loading file:  /Users/mzahn/data/SASSIE/SASSIE_examples/ocean_state_3D_day_mean/ocean_state_3D_day_mean.0005790000/ocean_state_3D_day_mean.0005797440.data
load_binary_array: loading file /Users/mzahn/data/SASSIE/SASSIE_examples/ocean_state_3D_day_mean/ocean_state_3D_day_mean.0005790000/ocean_state_3D_day_mean.0005797440.data
load_binary_array: data array shape  (180, 3800, 1080)
load_binary_array: data array type  >f4

... creating variab